## Load final saved model + labels

In [1]:
import os, json
import torch
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

MODEL_DIR = os.path.abspath("wavlm_ser_model")   # still needed for feat + labels (and as fallback config)
PTH_PATH = os.path.join("wavlm_ser_model.pth")

assert os.path.isfile(PTH_PATH), f".pth not found: {PTH_PATH}"
assert os.path.isdir(MODEL_DIR), f"Model folder not found (needed for feat/labels): {MODEL_DIR}"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 1) Load feature extractor 
feat = AutoFeatureExtractor.from_pretrained(MODEL_DIR, local_files_only=True)

# 2) Load labels + build mappings
with open(os.path.join(MODEL_DIR, "labels.json"), "r", encoding="utf-8") as f:
    labels = json.load(f)

label2id_pth = {lab:i for i, lab in enumerate(labels)}
id2label_pth = {i:lab for lab,i in label2id_pth.items()}


# 3) Rebuild model architecture EXACTLY (same backbone as training)
model = AutoModelForAudioClassification.from_pretrained(MODEL_DIR, local_files_only=True)

# 4) Load .pth weights
state = torch.load(PTH_PATH, map_location="cpu")
missing, unexpected = model.load_state_dict(state, strict=True)

# strict=True means it will error if mismatch
model = model.to(device).eval()

print("Loaded .pth weights into model:", PTH_PATH)
print("Labels:", labels)


Device: cpu
Loaded .pth weights into model: wavlm_ser_model.pth
Labels: ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']


## Choose the microphone device

In [2]:
import sounddevice as sd

print("INPUT devices:")
for i, d in enumerate(sd.query_devices()):
    if d["max_input_channels"] > 0:
        print(f"[{i}] {d['name']} | in_ch={d['max_input_channels']} | default_sr={d['default_samplerate']}")


INPUT devices:
[0] Microsoft Sound Mapper - Input | in_ch=2 | default_sr=44100.0
[1] Mikrofon (4- Fifine Microphone) | in_ch=1 | default_sr=44100.0
[2] Mikrofonarray (Intel® Smart Sou | in_ch=4 | default_sr=44100.0
[6] Primary Sound Capture Driver | in_ch=2 | default_sr=44100.0
[7] Mikrofon (4- Fifine Microphone) | in_ch=1 | default_sr=44100.0
[8] Mikrofonarray (Intel® Smart Sound Technologie) | in_ch=4 | default_sr=44100.0
[14] Mikrofon (4- Fifine Microphone) | in_ch=1 | default_sr=48000.0
[15] Mikrofonarray (Intel® Smart Sound Technologie) | in_ch=4 | default_sr=48000.0
[16] Mikrofonarray (Intel® Smart Sound Technologie Mikrofon) | in_ch=4 | default_sr=48000.0
[19] PC-Lautsprecher (Realtek HD Audio 2nd output with SST) | in_ch=2 | default_sr=48000.0
[22] PC-Lautsprecher (Realtek HD Audio output with SST) | in_ch=2 | default_sr=48000.0
[23] Mikrofon (Realtek HD Audio Mic input) | in_ch=2 | default_sr=44100.0
[24] Stereomix (Realtek HD Audio Stereo input) | in_ch=2 | default_sr=48000.0

## Working RMS check 

In [3]:
import numpy as np
import sounddevice as sd

MIC_INDEX = 14     # <-- your Fifine index that works
dev = sd.query_devices(MIC_INDEX)
SR = int(dev["default_samplerate"])
sd.default.device = (MIC_INDEX, None)

print("Using:", dev["name"], "| SR:", SR)

audio = sd.rec(int(SR*1.0), samplerate=SR, channels=1, dtype="float32")
sd.wait()
rms = float(np.sqrt(np.mean(audio[:,0]**2) + 1e-12))
print("RMS:", rms)


Using: Mikrofon (4- Fifine Microphone) | SR: 48000
RMS: 1.912239938870525e-05


## Live utterance detection (mic SR) → resample → predict

In [6]:
import numpy as np
import sounddevice as sd
import torch
import torchaudio
import time

MIC_INDEX = 14  # <-- your Fifine index that works
sd.default.device = (MIC_INDEX, None)

dev = sd.query_devices(MIC_INDEX)
MIC_SR = int(dev["default_samplerate"])
print("Using mic:", dev["name"], "| mic SR:", MIC_SR)

TARGET_SR = 16000  # model expects 16k

BLOCK_SEC = 0.25
BLOCK = int(MIC_SR * BLOCK_SEC)

END_SILENCE_SEC = 0.8
END_SILENCE_BLOCKS = max(1, int(END_SILENCE_SEC / BLOCK_SEC))
MIN_UTT_SEC = 0.6
MIN_UTT_LEN = int(MIC_SR * MIN_UTT_SEC)

MAX_SECONDS = 6.0
MAX_LEN = int(MIC_SR * MAX_SECONDS)

START_MULT = 3.0
END_MULT = 1.5

def rms(x):
    return float(np.sqrt(np.mean(x * x) + 1e-12))

def softmax(x):
    x = x - np.max(x)
    e = np.exp(x)
    return e / (np.sum(e) + 1e-12)

def resample_to_16k(audio, sr):
    if sr == TARGET_SR:
        return audio.astype(np.float32)
    w = torch.tensor(audio, dtype=torch.float32).unsqueeze(0)  # [1, T]
    w = torchaudio.functional.resample(w, sr, TARGET_SR)
    return w.squeeze(0).cpu().numpy().astype(np.float32)

def predict(audio_mic_sr):
    # normalize loudness
    peak = np.max(np.abs(audio_mic_sr)) + 1e-9
    audio_mic_sr = (audio_mic_sr / peak).astype(np.float32)

    # truncate very long utterances
    if len(audio_mic_sr) > MAX_LEN:
        audio_mic_sr = audio_mic_sr[-MAX_LEN:]

    audio_16k = resample_to_16k(audio_mic_sr, MIC_SR)

    inputs = feat([audio_16k], sampling_rate=TARGET_SR, padding=True, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits[0].detach().cpu().numpy()

    probs = softmax(logits)
    top = np.argsort(-probs)[:3]
    return [(labels[i], float(probs[i])) for i in top]

# Calibrate noise floor
print("Calibrating noise floor... stay silent for 1 second.")
calib = sd.rec(int(MIC_SR * 1.0), samplerate=MIC_SR, channels=1, dtype="float32")
sd.wait()
noise = rms(calib[:, 0])

start_th = max(0.005, noise * START_MULT)
end_th   = max(0.003, noise * END_MULT)

print(f"Noise RMS: {noise:.6f}")
print(f"Start_th: {start_th:.6f} | End_th: {end_th:.6f}")
print("\ LIVE MODE: say something with an emotion. Stop = Interrupt Kernel / Ctrl+C\n")

state = "idle"
utter = []
silence_blocks = 0

stream = sd.InputStream(samplerate=MIC_SR, channels=1, dtype="float32", blocksize=BLOCK)
stream.start()

try:
    while True:
        block, _ = stream.read(BLOCK)
        x = block[:, 0].copy()
        r = rms(x)

        if state == "idle":
            if r > start_th:
                state = "rec"
                utter = [x]
                silence_blocks = 0
                print("...speaking detected")
        else:
            utter.append(x)
            if r < end_th:
                silence_blocks += 1
            else:
                silence_blocks = 0

            if silence_blocks >= END_SILENCE_BLOCKS:
                audio = np.concatenate(utter, axis=0)

                if len(audio) < MIN_UTT_LEN:
                    print("Ignored very short sound.")
                else:
                    top3 = predict(audio)
                    print("PRED:", " | ".join([f"{lab}={p:.2f}" for lab, p in top3]))

                print("\n🎙️ LISTENING... say it again\n")
                state = "idle"
                utter = []
                silence_blocks = 0

        time.sleep(0.01)

except KeyboardInterrupt:
    print("\nStopped.")
finally:
    stream.stop()
    stream.close()


<>:77: SyntaxWarning: invalid escape sequence '\ '
<>:77: SyntaxWarning: invalid escape sequence '\ '
C:\Users\9826\AppData\Local\Temp\ipykernel_17604\1898270382.py:77: SyntaxWarning: invalid escape sequence '\ '
  print("\ LIVE MODE: say something with an emotion. Stop = Interrupt Kernel / Ctrl+C\n")


Using mic: Mikrofon (4- Fifine Microphone) | mic SR: 48000
Calibrating noise floor... stay silent for 1 second.
Noise RMS: 0.000019
Start_th: 0.005000 | End_th: 0.003000
\ LIVE MODE: say something with an emotion. Stop = Interrupt Kernel / Ctrl+C

...speaking detected
PRED: neutral=0.39 | surprise=0.23 | happy=0.19

🎙️ LISTENING... say it again

...speaking detected
PRED: neutral=0.33 | surprise=0.32 | happy=0.20

🎙️ LISTENING... say it again

...speaking detected
PRED: surprise=0.34 | neutral=0.28 | happy=0.21

🎙️ LISTENING... say it again

...speaking detected
PRED: surprise=0.66 | sad=0.14 | neutral=0.10

🎙️ LISTENING... say it again

...speaking detected
PRED: surprise=0.36 | neutral=0.30 | happy=0.19

🎙️ LISTENING... say it again

...speaking detected
PRED: happy=0.48 | fear=0.32 | sad=0.14

🎙️ LISTENING... say it again

...speaking detected
PRED: neutral=0.53 | surprise=0.21 | happy=0.12

🎙️ LISTENING... say it again

...speaking detected
PRED: happy=0.90 | fear=0.05 | neutral=0.